In [21]:
#First part
import os
import json
import codecs
#Get all the files in the directory
data_path = 'data/2015-03-18'
files = os.listdir(data_path)
names = []
titles = []
prices = []
#Parse through the files
for filename in files:
    file_path = data_path + '/' + filename
    with codecs.open(file_path,'r', encoding = 'utf-8') as file:
        prev_line = ''
        for line in file:
            if '<title>' in prev_line: #getting the title and artist
                titleline = line.lstrip()
                for i in range(len(titleline)):
                    c = titleline[i]
                    if c == ':': #colon seperates title and artist
                        break
                name = titleline[:i]
                title = titleline[i+2:-1]
                
            if 'Price realised' in prev_line: # getting the price
                priceline = line.lstrip()
                price = ''.join(c for c in priceline if c.isdigit())
                
            prev_line = line
        
        #Store the values for this html file
        names.append(name)
        titles.append(title)
        prices.append(price)

works = [{'title': title, 'price': price} for title, ]        
output = [{'artist': artist, 'works': title} for artist, title in zip(names,titles)]

output = json.dumps(output)
print(output)
print(prices)

[{"artist": "Rembrandt Harmensz. van Rijn", "works": "Christ at Emmaus: The smaller Plate"}, {"artist": "Louis Marcoussis", "works": "La Table"}, {"artist": "Joan Mir\u00f3", "works": "Femme et Chien devant la Lune"}, {"artist": "Pablo Picasso", "works": "Quatre Femmes nues et T\u00eate sculpt\u00e9e, from: La Suite Vollard"}, {"artist": "Marc Chagall", "works": "Self-Portrait"}]
['6875', '9200', '11250', '25000', '6000']
